In [145]:
## Importing libraries
import cv2
import os
import random
import itertools

import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [150]:
## Setting some importating global variables and constants
ROOT_FOLDER_TRAINING = "data_set/Training"
ROOT_FOLDER_TEST = "data_set/Test"

EPOCHS = 30
BATCH_SIZE = 32

In [106]:
## Renaming the folder names to lowercase and replacing the spaces for underscore(_)
def rename_folders(root_folder):
    for subdir, dirs, _ in os.walk(root_folder, topdown=False):
        for dir_name in dirs:
            new_dir_name = dir_name.replace(" ", "_").replace("-", "_").lower()  # Add more replacements if needed
            old_dir_path = os.path.join(subdir, dir_name)
            new_dir_path = os.path.join(subdir, new_dir_name)
            if old_dir_path != new_dir_path:
                os.rename(old_dir_path, new_dir_path)

rename_folders(ROOT_FOLDER_TRAINING)
rename_folders(ROOT_FOLDER_TEST)

In [107]:
## Getting the classes names
def classes(root_folder):
    classes = set()
    for subdir, dirs, _ in os.walk(root_folder):
        for dir_name in dirs:
            classes.add(dir_name.lower())
    return classes

classes = classes(ROOT_FOLDER_TRAINING)

In [139]:
## Reading the images from folders
def load_images_from_folder(root_folder):
    images = []
    for i in classes:
        path = os.path.join(root_folder, i)
        class_num = list(classes).index(i)

        for img in os.listdir(path):
            if img.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                img_array = cv2.imread(os.path.join(path, img))
                img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

                images.append([img_array, class_num])
    return images

images_training = load_images_from_folder(ROOT_FOLDER_TRAINING)
images_test = load_images_from_folder(ROOT_FOLDER_TEST)

In [140]:
## Creating the trainining and test arrays
random.shuffle(images_training)
random.shuffle(images_test)

X_train = []
Y_train = []

X_test = []
Y_test = []

def create_arrays_for_cnn(img_array, feature_array, class_array):
    for features, label in img_array:
        feature_array.append(features)
        class_array.append(label)

create_arrays_for_cnn(images_training, X_train, Y_train)
create_arrays_for_cnn(images_test, X_test, Y_test)

X_train = np.array(X_train)
X_test = np.array(X_test)

# Converting classes to categorial form
Y_train = to_categorical(Y_train, num_classes = 141)
Y_test = to_categorical(Y_test, num_classes = 141)


In [141]:
## Normalizing the images
X_train = X_train/255
X_test = X_test/255

In [153]:
## Split valdiation and train set
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [159]:
## Setting model architecture
model = Sequential()
model.add(Conv2D(filters=8, kernel_size=(3, 3), padding="Same", activation="relu", input_shape=(100, 100, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.35))

model.add(Conv2D(filters=16, kernel_size=(3, 3), padding="Same", activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.35))

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="Same", activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.35))
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(141, activation="softmax"))

optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)

# Assuming x_train and y_train are defined
datagen.fit(x_train)

# Model fitting using `fit` method
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
    epochs=EPOCHS,
    validation_data=(x_val, y_val),
    steps_per_epoch=x_train.shape[0] // BATCH_SIZE
)